In [223]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from lat_long import get_lat_from_zip,get_long_from_zip
import time
from joblib import dump, load
import pickle

path_model = "Eliza_XGB_Model.pkl"


In [184]:
def preprocess (df):

    # Nettoyer les type et sub type null
    df = df[df['Price'].notna()]
    
    df.loc[df["Type of sale"] == "public sale", 'Type of sale'] = None
    df = df[df['Type of sale'].notna()]
    df = df.drop(axis=1, labels="Type of sale")
    
    df.loc[df["Garden"] == 0, 'Garden Area'] = 0

    df.loc[df["Subtype of property"] == "villa", 'Type of property'] = "house"
    df.loc[df["Subtype of property"] == "loft", 'Type of property'] = "apartment"
    df.loc[df["Subtype of property"] == "mansion", 'Type of property'] = "house"
    df.loc[df["Subtype of property"] == "penthouse", 'Type of property'] = "apartment"
    df.loc[df["Subtype of property"] == "duplex", 'Type of property'] = "apartment"
    df.loc[df["Subtype of property"] == "investment property", 'Type of property'] = "house"

    df.loc[df["Url"].str.contains("immeuble-de-rapport-a-vendre"), 'Type of property'] = "house"
    df.loc[df["Url"].str.contains("immeuble-de-rapport-a-vendre"), 'Subtype of property'] = "investment property"

    df.loc[df["Number of rooms"] == 0, 'Number of rooms'] = None
    df.loc[((df["Number of rooms"].isnull()) & (df["Subtype of property"] == "studio")), 'Number of rooms'] = 0
    df.loc[((df["Number of rooms"].isnull()) & (df["Subtype of property"] == "student")), 'Number of rooms'] = 0
    df.loc[(df["Garden Area"].isnull()) & (df["Garden"] == 0), "Garden Area"] = 0
    df.loc[(df["Furnished"].isnull()), "Furnished"] = 0
    df.loc[((df["Subtype of property"].isnull()) & (df["Type of property"] == "apartment")), "Subtype of property"] = "apartment"
    df.loc[((df["Subtype of property"].isnull()) & (df["Type of property"] == "house")), "Subtype of property"] = "house"
    
    df = df.drop(axis=1, labels="Url")
    df = df.drop(axis=1, labels="Source")
    df = df.drop(axis=1, labels="Surface area of the plot of land")
    df = df.drop(axis=1, labels="Region")
    df = df.drop(axis=1, labels="Province")
    
    df = df.sort_values("Price", ascending=False)
    df = df[df['Price'] < 30000000]

    to_replace = {None : "good"}
    df["State of the building"] = df["State of the building"].replace(to_replace)
    df["Locality"] = df["Locality"].astype(str)
    df['lat'] = df["Locality"].apply(get_lat_from_zip)
    df['long'] = df["Locality"].apply(get_long_from_zip)
    df = df[df["lat"] != 0]
    df = df.drop(columns='Locality')
    df = pd.get_dummies(df, drop_first=False)

    return df

In [174]:
df = pd.read_csv("./Data/database.csv", index_col=0)
df_empty = df.iloc[0:0]
df = preprocess(df)
print(df.info())
print(df_empty.info())

(73510, 23)
(72506, 23)


/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 71465 entries, 19178 to 70072
Data columns (total 43 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Price                                    71465 non-null  float64
 1   Number of rooms                          70238 non-null  float64
 2   Area                                     61279 non-null  float64
 3   Fully equipped kitchen                   71465 non-null  float64
 4   Furnished                                71465 non-null  float64
 5   Open fire                                71465 non-null  float64
 6   Terrace                                  71465 non-null  float64
 7   Terrace Area                             23188 non-null  float64
 8   Garden                                   71465 non-null  float64
 9   Garden Area                              64848 non-null  float64
 10  Surface of the land                      3

In [160]:
model, df_knn = run(df)
np.save("fichierKNN6.data", df_knn)

start knn
fini en 695.14 secondes
start knn
fini en 0.08 secondes
(71465,)
(57172, 42)
(57172,)
(14293, 42)
(14293,)
0.9998373636974037 0.8366667151853359 0.16317064851206786


NameError: name 'path_model' is not defined

In [225]:
print(model)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
             grow_policy='lossguide', importance_type='gain',
             interaction_constraints='', learning_rate=0.1, max_bin=512,
             max_delta_step=0, max_depth=8, max_leaves=500, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=6000,
             n_jobs=6, num_parallel_tree=1, random_state=0, reg_alpha=5,
             reg_lambda=5, scale_pos_weight=1, subsample=1,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)


In [224]:
with open(path_model, 'wb') as file:
    pickle.dump(model, file)

In [ ]:
# Load the Model back from file
with open(path_model, 'rb') as file:
    model = pickle.load(file)

In [159]:
def run(df):
    df_knn = knn(df)
    model = train(df_knn)
    return model, df_knn

## Han

2 bed rooms, fully equipped kitchen, 2 small terraces, no garden, locality 1160
85 m2, Brussels 1160, fully equipped kitchen , fully renovated

## Olivier
1000 Brussels, 70m², 1 bedroom, fully equipped kitchen, no terrace/garden. it's a corner appartement (2 facades)

In [206]:
to_predict = {"Number of rooms":[2], "Fully equipped kitchen": [1], "Terrace": [1], "Garden": [0], "Swimming pool":[0],
             "Locality":[1160], "Area": [85], "State of the building": ["good"], "Type of property": ["apartment"]}

In [214]:
to_predict = {"Number of rooms":[1], "Fully equipped kitchen": [1], "Terrace": [0], "Garden": [1], "Swimming pool":[0],
             "Locality":[1000], "Area": [70], "State of the building": ["good"], "Type of property": ["apartment"],
              "Number of facades": 2}

In [218]:
to_predict = {"Number of rooms":[2], "Fully equipped kitchen": [1], "Terrace": [1], "Terrace Area": 30, "Garden": [1],
              "Swimming pool":[0], "Locality":[1200], "Area": [120], "State of the building": ["good"],
              "Type of property": ["apartment"], "Number of facades": 2}

In [219]:
df_to_predict = pd.DataFrame.from_dict(to_predict)
df_to_predict['Price'] = 0
df_to_predict["Type of sale"] = "predict"
df_to_predict["Url"] = "predict"
df_to_predict = df_empty.append(df_to_predict)

df_to_predict = preprocess(df_to_predict)

df_empty_long = df.iloc[0:0]
df_to_predict = df_empty_long.append(df_to_predict)

df_to_predict2 = df_to_predict.drop("Price", axis=1)
df_to_predict2.shape
prediction = model.predict(df_to_predict2)
print(prediction, '€')


[418201.47] €


## Imputer

In [136]:
def knn(data_input):
    start = time.perf_counter()
    print("start knn")
    imputer = KNNImputer(n_neighbors = 10)
    df_knn = imputer.fit_transform(data_input)
    #  np.save("nomdufichier6.data", df_knn)

    finish = time.perf_counter()
    print(f"fini en {round(finish-start, 2)} secondes")
    return  df_knn

In [144]:
def add_to_knn(row=True):
    print("start add_knn")
    if type(row) is pd.DataFrame:
        row = row.to_numpy()
    df_knn = np.load("nomdufichier6.data.npy")
    print(df_knn.size)
    np.vstack((df_knn, row))
    print(df_knn.size)
    print(type(df_knn))
    return df_knn

In [ ]:
# df_knn = np.load("nomdufichier6.data.npy")
# df = np.load("nomdufichier.data.npy")


(57172, 39)
(57172,)
(14293, 39)
(14293,)


## XGBRegressor - 83.6%

In [133]:
def train(df_train):
    df_knn = knn(df_train)
    np.save("nomdufichier6.data", df_knn)

    y = df_knn[:,0] #  .reshape(-1,1)
    X = df_knn[:,1:]
    y = y.T
    print(y.shape)

    # Splitting data into train and test split
    x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=41, test_size=0.2)
    print(x_train.shape)
    print(y_train.shape)
    print(x_test.shape)
    print(y_test.shape)

    model = xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=8, grow_policy = 'lossguide', max_leaves = 500,
                        max_bin = 512, reg_alpha = 5, reg_lambda = 5,
                        n_estimators = 6000, learning_rate=0.1, tree_method = 'gpu_hist')

    model.fit(x_train,y_train)
    print(model.score(x_train, y_train), model.score(x_test, y_test), (model.score(x_train, y_train)- model.score(x_test, y_test)))
    return model

0.9998373636974037 0.8366667151853359 0.16317064851206786


model = xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=8, #num_parallel_tree = 3, 
                        reg_alpha = 5, # max_bin = 1024,
                        n_estimators = 4000, learning_rate=0.1, tree_method = 'gpu_hist')
0.9998132635082202 0.851047719752367 0.14876554375585316


model = xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=8, #num_parallel_tree = 3, 
                        max_bin = 1024, reg_alpha = 5,
                        n_estimators = 4000, learning_rate=0.1, tree_method = 'gpu_hist')
0.999765113520287 0.8505623053783662 0.14920280814192077

model = xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=8, #num_parallel_tree = 3, 
                        max_bin = 512, reg_lambda = 10,
                        n_estimators = 4000, learning_rate=0.1, tree_method = 'gpu_hist')
0.9987876063930322 0.8566091475621213 0.14217845883091096

model = xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=8, #num_parallel_tree = 3, 
                        max_bin = 512, reg_alpha = 10, reg_lambda = 10,
                        n_estimators = 4000, learning_rate=0.1, tree_method = 'gpu_hist')
0.998762287541839 0.8553727356152264 0.14338955192661262

#### model = xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=8,
                        max_bin = 512, reg_alpha = 5, reg_lambda = 5,
                        n_estimators = 4000, learning_rate=0.1, tree_method = 'gpu_hist')
0.9993692375342816 0.8571282283314186 0.1422410092028631

model = xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=8,
                        max_bin = 512, reg_alpha = 2,
                        n_estimators = 4000, learning_rate=0.1, tree_method = 'gpu_hist')
0.9997917613166557 0.855281945931566 0.14450981538508967

model = xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=8,
                        max_bin = 512,
                        n_estimators = 4000, learning_rate=0.1, tree_method = 'gpu_hist')
0.9997983720288148 0.8567727264217655 0.14302564560704933

model = xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=8, num_parallel_tree = 3, 
                        n_estimators = 4000, learning_rate=0.1, tree_method = 'gpu_hist')
0.99980853482703 0.8521896845089079 0.14761885031812216


model =xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=8,
                        n_estimators = 4000, learning_rate=0.1, tree_method = 'gpu_hist')
0.999802967794843 0.8516998776901281 0.1481030901047149


model =xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=8, grow_policy = 'lossguide', max_leaves = 500,
                        num_parallel_tree = 5, max_bin = 512,
                        n_estimators = 4000, learning_rate=0.05, tree_method = 'gpu_hist')
0.9983446665082456 0.8563764861118395 0.14196818039640613

model =xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=8, grow_policy = 'lossguide', max_leaves = 500,
                        num_parallel_tree = 5, max_bin = 512,
                        n_estimators = 6000, learning_rate=0.1, tree_method = 'gpu_hist')
0.9999623287317445 0.8567858238632804 0.14317650486846412     # 30 minutes

model =xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=8, grow_policy = 'lossguide', max_leaves = 20,
                        n_estimators = 6000, learning_rate=0.1, tree_method = 'gpu_hist')
0.9997593543466164 0.8502793314915857 0.1494800228550307

model =xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=8, grow_policy = 'lossguide', max_leaves = 20,
                        n_estimators = 6000, learning_rate=0.1, tree_method = 'gpu_hist')
0.9931133400714447 0.8468483073335407 0.14626503273790403

model =xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=8, grow_policy = 'lossguide', max_leaves = 5,
                        n_estimators = 6000, learning_rate=0.1, tree_method = 'gpu_hist')
0.9526180694530272 0.8340390212150617 0.1185790482379655

model =xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=8,
                        n_estimators = 6000, learning_rate=0.1, tree_method = 'gpu_hist')
0.9999628823878924 0.8517063326995244 0.148256549688368


model =xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=9,
                        n_estimators = 6000, learning_rate=0.1, tree_method = 'gpu_hist')
0.999992674758225 0.8435944093630555 0.15639826539516943


model =xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=7,
                        n_estimators = 6000, learning_rate=0.1, tree_method = 'gpu_hist')
0.9996491336382041 0.8476513430580268 0.15199779058017726

model =xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=6,
                        n_estimators = 6000, learning_rate=0.1, tree_method = 'gpu_hist')
0.9979864533735407 0.8462007798620378 0.15178567351150296

model =xgb.XGBRegressor(random_state=0, n_jobs=6, max_depth=5,
                        n_estimators = 6000, learning_rate=0.1, tree_method = 'gpu_hist')
0.9927420904117953 0.8445279867247475 0.1482141036870478


                        n_estimators = 6000, learning_rate=0.1, tree_method = 'gpu_hist')
0.9979864533735407 0.8462007798620378 0.15178567351150296

model =xgb.XGBRegressor(n_estimators = 4000, random_state=0, learning_rate=0.1, tree_method = 'gpu_hist')
0.9959100162368746 0.8459608583920194 0.14994915784485519

model =xgb.XGBRegressor(n_estimators = 4000, random_state=0, learning_rate=0.08, tree_method = 'gpu_hist',
                       n_jobs=6)
0.9941049256512487 0.8433765141647601 0.15072841148648863

model =xgb.XGBRegressor(n_estimators = 4000, random_state=0, learning_rate=0.05, tree_method = 'gpu_hist')
0.9893983015007012 0.8423624230323355 0.14703587846836563

model =xgb.XGBRegressor(n_estimators = 4000, random_state=0, learning_rate=0.1, tree_method = 'hist')
0.9958906268886126 0.8450341622976418 0.1508564645909708

model =xgb.XGBRegressor(n_estimators = 2000, random_state=0, learning_rate=0.2, tree_method = 'hist')
0.9960097119421771 0.8427440577385367 0.15326565420364047

model =xgb.XGBRegressor(n_estimators = 1000, random_state=0, learning_rate=0.2, tree_method = 'hist')
0.989814872660948 0.84184660889742 0.147968263763528

model =xgb.XGBRegressor(n_estimators = 500, random_state=0, learning_rate=0.2, tree_method = 'hist')
0.9791561680982418 0.8400372929358884 0.13911887516235344

model =xgb.XGBRegressor(n_estimators = 500, random_state=0, learning_rate=0.1, tree_method = 'hist')
0.9632291677458388 0.8374924645823841 0.12573670316345464

model =xgb.XGBRegressor(random_state=0, learning_rate=0.1, tree_method = 'hist')
0.9117170222749945 0.8149151142209548 0.09680190805403976

model =xgb.XGBRegressor(random_state=0, learning_rate=0.1)
0.9156693866898443 0.8133889759099304 0.1022804107799139

model =xgb.XGBRegressor (random_state=0)
0.9477787390919068 0.808287698716571 0.13949104037533577

model =xgb.XGBRegressor(random_state=0, learning_rate=1)
0.968737930521364 0.7390980419613071 0.2296398885600569

model =xgb.XGBRegressor(random_state=0, learning_rate=0.01)
0.5825861847357299 0.5201486702748421 0.062437514460887744

model =xgb.XGBRegressor(random_state=0, learning_rate=0.5)
0.9601038139096176 0.8113167204440415 0.14878709346557617